In [1]:
# === V4: OPTIMIZED BASELINE + 2000 TERMS ===
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

CONFIG = {
    "n_terms": 2000,  # INCREASED from 1500
    "paths": {
        "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
        "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
        "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
        "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
        "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
        "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
    }
}

# LOAD DATA
print("Loading data...")
train_emb = np.load(CONFIG["paths"]["train_emb"]).astype(np.float32)
train_ids = np.load(CONFIG["paths"]["train_ids"])
test_emb = np.load(CONFIG["paths"]["test_emb"]).astype(np.float32)
test_ids = np.load(CONFIG["paths"]["test_ids"])

print(f"Train: {train_emb.shape}, Test: {test_emb.shape}")

# Normalize
mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

# Load terms
terms_df = pd.read_csv(CONFIG["paths"]["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

# MODEL (Same as your 0.195 winner)
class SimpleModel(nn.Module):
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class SimpleData(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return (self.X[i], self.y[i]) if self.y is not None else self.X[i]

# TRAIN
def train_aspect(aspect_char, aspect_name):
    print(f"\n>>> Training {aspect_name} ({aspect_char})...")
    
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:CONFIG["n_terms"]].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    num_classes = len(top_terms)
    print(f"Using {num_classes} GO terms")
    
    label_matrix = np.zeros((len(train_ids), num_classes), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    
    for _, row in tqdm(relevant.iterrows(), total=len(relevant), desc="Labels"):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    ds = SimpleData(train_emb, label_matrix)
    loader = DataLoader(ds, batch_size=256, shuffle=True, num_workers=2)
    
    model = SimpleModel(1280, num_classes).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.BCEWithLogitsLoss()
    
    for epoch in range(10):
        model.train()
        total = 0
        for x, y in tqdm(loader, desc=f"Epoch {epoch+1}", leave=False):
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            loss = loss_fn(model(x), y)
            loss.backward()
            opt.step()
            total += loss.item()
        print(f"Epoch {epoch+1} Loss: {total/len(loader):.4f}")
    
    torch.save(model.state_dict(), f"model_v4_{aspect_char}.pth")
    return top_terms

# Train all 3
print("\n=== TRAINING ===")
results = {}
for char, name in [('F', 'Function'), ('P', 'Process'), ('C', 'Component')]:
    results[char] = train_aspect(char, name)

print("\n Training Complete!")

# PREDICT
print("\n=== PREDICTING ===")
with open("submission_v4_2000terms.tsv", "w") as f:
    for char in ['F', 'P', 'C']:
        print(f"Predicting {char}...")
        terms = results[char]
        num_classes = len(terms)
        
        model = SimpleModel(1280, num_classes).to(device)
        model.load_state_dict(torch.load(f"model_v4_{char}.pth"))
        model.eval()
        
        loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
        preds = []
        with torch.no_grad():
            for x in tqdm(loader):
                preds.append(torch.sigmoid(model(x.to(device))).cpu().numpy())
        
        all_preds = np.vstack(preds)
        
        for i, pid in enumerate(tqdm(test_ids)):
            top_idx = np.argpartition(all_preds[i], -70)[-70:]
            for idx in top_idx:
                if all_preds[i, idx] > 0.01:
                    f.write(f"{pid}\t{terms[idx]}\t{all_preds[i, idx]:.3f}\n")
        
        del model, preds
        gc.collect()

print("\n Predictions Complete!")


Device: cuda
Loading data...
Train: (82404, 1280), Test: (224309, 1280)

=== TRAINING ===

>>> Training Function (F)...
Using 2000 GO terms


Labels: 100%|██████████| 116189/116189 [00:04<00:00, 24309.90it/s]


Epoch 1 Loss: 0.0164


Epoch 2 Loss: 0.0036


Epoch 3 Loss: 0.0032


...

Predicting P...


100%|██████████| 224309/224309 [00:37<00:00, 5966.39it/s]


Predicting C...


100%|██████████| 224309/224309 [00:40<00:00, 5486.43it/s]



 Predictions Complete!


In [2]:
# === GRAPH PROPAGATION ===
!pip install obonet networkx -q

import obonet
from tqdm import tqdm

print("Loading GO Graph...")
graph = obonet.read_obo(CONFIG["paths"]["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

print("Loading predictions...")
sub = {}
with open("submission_v4_2000terms.tsv") as f:
    for line in tqdm(f):
        p, t, s = line.strip().split("\t")
        if p not in sub: sub[p] = {}
        sub[p][t] = float(s)

print("Propagating...")
with open("submission.tsv", "w") as f:
    for pid, preds in tqdm(sub.items()):
        final = preds.copy()
        q = list(preds.keys())
        visited = set(q)
        
        while q:
            term = q.pop(0)
            for par in parent_map.get(term, []):
                if final.get(par, 0) < final[term]:
                    final[par] = final[term]
                    if par not in visited:
                        q.append(par)
                        visited.add(par)
        
        for t, s in sorted(final.items(), key=lambda x: -x[1])[:70]:
            if s > 0.001:
                f.write(f"{pid}\t{t}\t{s:.3f}\n")

print("\n✅ submission.tsv ready! Download and submit.")


Loading GO Graph...
Loading predictions...


17114772it [00:15, 1081042.36it/s]


Propagating...


100%|██████████| 224309/224309 [01:03<00:00, 3547.35it/s]


✅ submission.tsv ready! Download and submit.


In [3]:
# === ENSEMBLE: 3 MODELS WITH DIFFERENT SEEDS ===
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CONFIG = {
    "n_terms": 1500,  # KEEP AT 1500
    "n_models": 3,     # Train 3 different models
    "paths": {
        "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
        "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
        "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
        "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
        "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
        "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
    }
}

# LOAD DATA
print("Loading...")
train_emb = np.load(CONFIG["paths"]["train_emb"]).astype(np.float32)
train_ids = np.load(CONFIG["paths"]["train_ids"])
test_emb = np.load(CONFIG["paths"]["test_emb"]).astype(np.float32)
test_ids = np.load(CONFIG["paths"]["test_ids"])

mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

terms_df = pd.read_csv(CONFIG["paths"]["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

# MODEL
class SimpleModel(nn.Module):
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class SimpleData(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return (self.X[i], self.y[i]) if self.y is not None else self.X[i]

# TRAIN MULTIPLE MODELS
def train_ensemble(aspect_char, aspect_name):
    print(f"\n>>> Training {aspect_name} Ensemble...")
    
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:CONFIG["n_terms"]].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    
    label_matrix = np.zeros((len(train_ids), len(top_terms)), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    for _, row in tqdm(relevant.iterrows(), total=len(relevant)):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    # Train N models with different seeds
    for model_idx in range(CONFIG["n_models"]):
        print(f"\n  Model {model_idx+1}/{CONFIG['n_models']}...")
        
        # Set seed for reproducibility
        torch.manual_seed(42 + model_idx)
        np.random.seed(42 + model_idx)
        
        ds = SimpleData(train_emb, label_matrix)
        loader = DataLoader(ds, batch_size=256, shuffle=True)
        
        model = SimpleModel(1280, len(top_terms)).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss()
        
        for epoch in range(10):
            model.train()
            total = 0
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                opt.zero_grad()
                loss = loss_fn(model(x), y)
                loss.backward()
                opt.step()
                total += loss.item()
            if epoch % 3 == 0:
                print(f"    Epoch {epoch+1}: {total/len(loader):.4f}")
        
        torch.save(model.state_dict(), f"model_ens_{aspect_char}_{model_idx}.pth")
    
    return top_terms

# Train ensembles
results = {}
for char, name in [('F', 'Function'), ('P', 'Process'), ('C', 'Component')]:
    results[char] = train_ensemble(char, name)

# PREDICT (Average all models)
print("\n=== ENSEMBLE PREDICTION ===")
with open("submission_ensemble.tsv", "w") as f:
    for char in ['F', 'P', 'C']:
        print(f"\nEnsembling {char}...")
        terms = results[char]
        
        # Collect predictions from all models
        all_model_preds = []
        for model_idx in range(CONFIG["n_models"]):
            model = SimpleModel(1280, len(terms)).to(device)
            model.load_state_dict(torch.load(f"model_ens_{char}_{model_idx}.pth"))
            model.eval()
            
            loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
            preds = []
            with torch.no_grad():
                for x in loader:
                    preds.append(torch.sigmoid(model(x.to(device))).cpu().numpy())
            
            all_model_preds.append(np.vstack(preds))
            del model
        
        # AVERAGE predictions
        final_preds = np.mean(all_model_preds, axis=0)
        
        for i, pid in enumerate(tqdm(test_ids)):
            top_idx = np.argpartition(final_preds[i], -70)[-70:]
            for idx in top_idx:
                if final_preds[i, idx] > 0.01:
                    f.write(f"{pid}\t{terms[idx]}\t{final_preds[i, idx]:.3f}\n")
        
        del all_model_preds
        gc.collect()

print("\n✅ Ensemble complete!")


Loading...

>>> Training Function Ensemble...


100%|██████████| 112061/112061 [00:04<00:00, 25618.54it/s]



  Model 1/3...
    Epoch 1: 0.0174
    Epoch 4: 0.0037
    Epoch 7: 0.0032
    Epoch 10: 0.0029

  Model 2/3...
    Epoch 1: 0.0174
    Epoch 4: 0.0037
    Epoch 7: 0.0032
    Epoch 10: 0.0029

  Model 3/3...
    Epoch 1: 0.0172
    Epoch 4: 0.0037
    Epoch 7: 0.0031
    Epoch 10: 0.0029

>>> Training Process Ensemble...


100%|██████████| 143554/143554 [00:05<00:00, 25453.67it/s]



  Model 1/3...
    Epoch 1: 0.0212
    Epoch 4: 0.0070
    Epoch 7: 0.0064
    Epoch 10: 0.0061

  Model 2/3...
    Epoch 1: 0.0212
    Epoch 4: 0.0070
    Epoch 7: 0.0064
    Epoch 10: 0.0061

  Model 3/3...
    Epoch 1: 0.0210
    Epoch 4: 0.0070
    Epoch 7: 0.0064
    Epoch 10: 0.0061

>>> Training Component Ensemble...


...


Ensembling P...


100%|██████████| 224309/224309 [00:39<00:00, 5610.50it/s]



Ensembling C...


100%|██████████| 224309/224309 [00:40<00:00, 5490.16it/s]


✅ Ensemble complete!


In [4]:
!pip install obonet -q
import obonet
graph = obonet.read_obo(CONFIG["paths"]["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

sub = {}
with open("submission_ensemble.tsv") as f:
    for line in f:
        p, t, s = line.strip().split("\t")
        if p not in sub: sub[p] = {}
        sub[p][t] = float(s)

with open("submissionfinal.tsv", "w") as f:
    for pid, preds in tqdm(sub.items()):
        final = preds.copy()
        q = list(preds.keys())
        while q:
            term = q.pop(0)
            for par in parent_map.get(term, []):
                if final.get(par, 0) < final[term]:
                    final[par] = final[term]
                    if par not in preds: q.append(par)
        for t, s in sorted(final.items(), key=lambda x: -x[1])[:70]:
            if s > 0.001:
                f.write(f"{pid}\t{t}\t{s:.3f}\n")


100%|██████████| 224309/224309 [01:08<00:00, 3268.21it/s]


In [5]:
# === DIVERSE ARCHITECTURE ENSEMBLE ===
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CONFIG = {
    "n_terms": 1500,
    "paths": {
        "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
        "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
        "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
        "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
        "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
        "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
    }
}

# LOAD DATA
print("Loading...")
train_emb = np.load(CONFIG["paths"]["train_emb"]).astype(np.float32)
train_ids = np.load(CONFIG["paths"]["train_ids"])
test_emb = np.load(CONFIG["paths"]["test_emb"]).astype(np.float32)
test_ids = np.load(CONFIG["paths"]["test_ids"])

mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

terms_df = pd.read_csv(CONFIG["paths"]["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

# THREE DIFFERENT ARCHITECTURES
class Model_A(nn.Module):  # Original (2-layer, 512)
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class Model_B(nn.Module):  # Deeper (3-layer)
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class Model_C(nn.Module):  # Wider + BatchNorm
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, n_class)
        )
    def forward(self, x): return self.net(x)

class SimpleData(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return (self.X[i], self.y[i]) if self.y is not None else self.X[i]

# TRAIN
def train_diverse_ensemble(aspect_char, aspect_name):
    print(f"\n>>> Training {aspect_name} Diverse Ensemble...")
    
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:CONFIG["n_terms"]].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    
    label_matrix = np.zeros((len(train_ids), len(top_terms)), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    for _, row in tqdm(relevant.iterrows(), total=len(relevant)):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    models = [Model_A, Model_B, Model_C]
    model_names = ['Simple', 'Deep', 'Wide+BN']
    
    for idx, (ModelClass, name) in enumerate(zip(models, model_names)):
        print(f"\n  Training {name}...")
        torch.manual_seed(42 + idx)
        
        ds = SimpleData(train_emb, label_matrix)
        loader = DataLoader(ds, batch_size=256, shuffle=True, num_workers=2)
        
        model = ModelClass(1280, len(top_terms)).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss()
        
        for epoch in range(10):
            model.train()
            total = 0
            for x, y in tqdm(loader, leave=False):
                x, y = x.to(device), y.to(device)
                opt.zero_grad()
                loss = loss_fn(model(x), y)
                loss.backward()
                opt.step()
                total += loss.item()
            if epoch % 3 == 0:
                print(f"    Epoch {epoch+1}: {total/len(loader):.4f}")
        
        torch.save(model.state_dict(), f"model_diverse_{aspect_char}_{idx}.pth")
    
    return top_terms

# Train
results = {}
for char, name in [('F', 'Function'), ('P', 'Process'), ('C', 'Component')]:
    results[char] = train_diverse_ensemble(char, name)

# PREDICT
print("\n=== DIVERSE ENSEMBLE PREDICTION ===")
with open("submission_diverse.tsv", "w") as f:
    for char in ['F', 'P', 'C']:
        print(f"\nEnsembling {char}...")
        terms = results[char]
        models = [Model_A, Model_B, Model_C]
        
        all_preds = []
        for idx, ModelClass in enumerate(models):
            model = ModelClass(1280, len(terms)).to(device)
            model.load_state_dict(torch.load(f"model_diverse_{char}_{idx}.pth"))
            model.eval()
            
            loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
            preds = []
            with torch.no_grad():
                for x in tqdm(loader, leave=False):
                    preds.append(torch.sigmoid(model(x.to(device))).cpu().numpy())
            
            all_preds.append(np.vstack(preds))
            del model
        
        # Average
        final_preds = np.mean(all_preds, axis=0)
        
        for i, pid in enumerate(tqdm(test_ids)):
            top_idx = np.argpartition(final_preds[i], -70)[-70:]
            for idx in top_idx:
                if final_preds[i, idx] > 0.01:
                    f.write(f"{pid}\t{terms[idx]}\t{final_preds[i, idx]:.3f}\n")
        
        del all_preds
        gc.collect()

print("\n✅ Done!")


Loading...

>>> Training Function Diverse Ensemble...


100%|██████████| 112061/112061 [00:04<00:00, 25478.05it/s]



  Training Simple...


    Epoch 1: 0.0174


    Epoch 4: 0.0037


...


Ensembling P...


100%|██████████| 224309/224309 [00:40<00:00, 5497.43it/s]



Ensembling C...


100%|██████████| 224309/224309 [00:41<00:00, 5459.25it/s]



✅ Done!


In [6]:
!pip install obonet -q
import obonet
graph = obonet.read_obo(CONFIG["paths"]["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

sub = {}
with open("submission_diverse.tsv") as f:
    for line in f:
        p, t, s = line.strip().split("\t")
        if p not in sub: sub[p] = {}
        sub[p][t] = float(s)

with open("submissionSimple+Deep+Wide.tsv", "w") as f:
    for pid, preds in tqdm(sub.items()):
        final = preds.copy()
        q = list(preds.keys())
        while q:
            term = q.pop(0)
            for par in parent_map.get(term, []):
                if final.get(par, 0) < final[term]:
                    final[par] = final[term]
                    if par not in preds: q.append(par)
        for t, s in sorted(final.items(), key=lambda x: -x[1])[:70]:
            if s > 0.001:
                f.write(f"{pid}\t{t}\t{s:.3f}\n")


100%|██████████| 224309/224309 [01:21<00:00, 2761.17it/s]
